 # Evaluation des modèles de `detectron2`

Ce notebook a pour but d'évaluer les modèles de détection d'objets et de segmentation d'instance issus de [detectron2](https://github.com/facebookresearch/detectron2). L'évaluation est réalisée sur le jeu de validation de 2017 de [COCO](http://cocodataset.org/#home), nommé `val2017`.

# Préparation de l'environnement d'exécution

## Installation des différentes librairies

Voir https://colab.research.google.com/drive/16jcaJoc6bCFAQ96jDe2HwtXj7BMD_-m5

In [1]:
# install dependencies:
# (use +cu100 because colab is on CUDA 10.0)
!pip install -U torch==1.4+cu100 torchvision==0.5+cu100 -f https://download.pytorch.org/whl/torch_stable.html 
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
torch.__version__
!gcc --version
# opencv is pre-installed on colab

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 723.9MB 24kB/s 
     |████████████████████████████████| 4.1MB 47.4MB/s 
  Found existing installation: torch 1.4.0
    Uninstalling torch-1.4.0:
      Successfully uninstalled torch-1.4.0
  Found existing installation: torchvision 0.5.0
    Uninstalling torchvision-0.5.0:
      Successfully uninstalled torchvision-0.5.0
     |████████████████████████████████| 276kB 32.4MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp36-cp36m-linux_x86_64.whl size=44074 sha256=de50fa21c21f1bdd0485296b5d17869c69476fe12a1c5b8132d877247f723407
  Stored in directory: /root/.cache/pip/wheels/ad/56/bc/1522f864feb2a358ea6f1a92b4798d69ac783a28e80567a18b
Successfully built pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-ess88nlu
  Running command git

In [2]:
# install detectron2:
!git clone https://github.com/facebookresearch/detectron2 detectron2_repo
!pip install -e detectron2_repo

Cloning into 'detectron2_repo'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 2684 (delta 0), reused 1 (delta 0), pack-reused 2681
Receiving objects: 100% (2684/2684), 1.91 MiB | 1.46 MiB/s, done.
Resolving deltas: 100% (1817/1817), done.
Obtaining file:///content/detectron2_repo
     |████████████████████████████████| 61kB 9.7MB/s 
  Created wheel for fvcore: filename=fvcore-0.1.dev200204-cp36-none-any.whl size=34351 sha256=105bb5b0a997f7f4b174009b40679a8fb199da38f7540cbb69e7fc413de12fb6
  Stored in directory: /root/.cache/pip/wheels/3e/51/21/78269ec63af8083e6e1eb8d8cfd349d24fe633423a83c21c17
Successfully built fvcore
  Found existing installation: tqdm 4.28.1
    Uninstalling tqdm-4.28.1:
      Successfully uninstalled tqdm-4.28.1
  Running setup.py develop for detectron2


In [0]:
# You may need to restart your runtime prior to this, to let your installation take effect
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import cv2
import random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

## GPU

On s'assure que le GPU est bien un Nvidia Tesla T4 (si aucun GPU n'est trouvé, il faut activer l'accélération matérielle dans Exécution > Modifier le type d'exécution).

In [2]:
!nvidia-smi

Tue Feb  4 22:21:31 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.48.02    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

## Connexion à Google Drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Copie du dataset et des annotations sur la vm :

In [0]:
!cp -r /content/drive/My\ Drive/data_redev/coco /content/detectron2_repo/datasets/

# Evaluation

In [0]:
!mkdir detectron2_repo/output

## Faster R-CNN

### R50-FPN

In [0]:
!cd detectron2_repo && python tools/train_net.py \
	--config-file configs/COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml \
	--eval-only MODEL.WEIGHTS detectron2://COCO-Detection/faster_rcnn_R_50_FPN_3x/137849458/model_final_280758.pkl

Command Line Args: Namespace(config_file='configs/COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml', dist_url='tcp://127.0.0.1:49152', eval_only=True, machine_rank=0, num_gpus=1, num_machines=1, opts=['MODEL.WEIGHTS', 'detectron2://COCO-Detection/faster_rcnn_R_50_FPN_3x/137849458/model_final_280758.pkl'], resume=False)
[01/30 20:33:05 detectron2]: Rank of current process: 0. World size: 1
[01/30 20:33:06 detectron2]: Environment info:
------------------------  ---------------------------------------------------------------
sys.platform              linux
Python                    3.6.9 (default, Nov  7 2019, 10:44:02) [GCC 8.3.0]
numpy                     1.17.5
detectron2                0.1 @/content/detectron2_repo/detectron2
detectron2 compiler       GCC 7.4
detectron2 CUDA compiler  10.0
detectron2 arch flags     sm_75
DETECTRON2_ENV_MODULE     <not set>
PyTorch                   1.4.0+cu100 @/usr/local/lib/python3.6/dist-packages/torch
PyTorch debug build       False
CUDA available    

Sauvegarde des résultats sur Google Drive :

In [0]:
!cp detectron2_repo/output/log.txt /content/drive/My\ Drive/res_eval_redev/faster_rcnn_r50_fpn.txt

### X101-FPN

In [0]:
!cd detectron2_repo && python tools/train_net.py \
	--config-file configs/COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml \
	--eval-only MODEL.WEIGHTS detectron2://COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x/139173657/model_final_68b088.pkl

Command Line Args: Namespace(config_file='configs/COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml', dist_url='tcp://127.0.0.1:49152', eval_only=True, machine_rank=0, num_gpus=1, num_machines=1, opts=['MODEL.WEIGHTS', 'detectron2://COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x/139173657/model_final_68b088.pkl'], resume=False)
[01/30 20:55:56 detectron2]: Rank of current process: 0. World size: 1
[01/30 20:55:56 detectron2]: Environment info:
------------------------  ---------------------------------------------------------------
sys.platform              linux
Python                    3.6.9 (default, Nov  7 2019, 10:44:02) [GCC 8.3.0]
numpy                     1.17.5
detectron2                0.1 @/content/detectron2_repo/detectron2
detectron2 compiler       GCC 7.4
detectron2 CUDA compiler  10.0
detectron2 arch flags     sm_75
DETECTRON2_ENV_MODULE     <not set>
PyTorch                   1.4.0+cu100 @/usr/local/lib/python3.6/dist-packages/torch
PyTorch debug build       False
CUDA

Sauvegarde des résultats sur Google Drive :

In [0]:
!cp detectron2_repo/output/log.txt /content/drive/My\ Drive/res_eval_redev/faster_rcnn_x101_fpn.txt

## RetinaNet

In [0]:
!cd detectron2_repo && python tools/train_net.py \
	--config-file configs/COCO-Detection/retinanet_R_101_FPN_3x.yaml \
	--eval-only MODEL.WEIGHTS detectron2://COCO-Detection/retinanet_R_101_FPN_3x/138363263/model_final_59f53c.pkl > output/retinanet_r101.txt

Loading config configs/COCO-Detection/../Base-RetinaNet.yaml with yaml.unsafe_load. Your machine may be at risk if the file contains malicious content.
model_final_59f53c.pkl: 228MB [00:14, 16.0MB/s]               


In [0]:
!cp detectron2_repo/output/retinanet_r101.txt /content/drive/My\ Drive/res_eval_redev

## Mask R-CNN

### R50-FPN

In [0]:
!cd detectron2_repo && python tools/train_net.py \
	--config-file configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml \
	--eval-only MODEL.WEIGHTS detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl > output/mask_rcnn_r50_fpn.txt

model_final_f10217.pkl: 178MB [00:10, 16.5MB/s]               


In [0]:
!cp detectron2_repo/output/mask_rcnn_r50_fpn.txt /content/drive/My\ Drive/res_eval_redev

### X101-FPN

In [0]:
!cd detectron2_repo && python tools/train_net.py \
	--config-file configs/COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml \
	--eval-only MODEL.WEIGHTS detectron2://COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x/139653917/model_final_2d9806.pkl > output/mask_rcnn_x101_fpn.txt

model_final_2d9806.pkl: 431MB [00:20, 21.1MB/s]               


In [0]:
!cp detectron2_repo/output/mask_rcnn_x101_fpn.txt /content/drive/My\ Drive/res_eval_redev

## Cascade Mask R-CNN

In [0]:
!cd detectron2_repo && python tools/train_net.py \
	--config-file configs/Misc/cascade_mask_rcnn_R_50_FPN_3x.yaml \
	--eval-only MODEL.WEIGHTS detectron2://Misc/cascade_mask_rcnn_R_50_FPN_3x/144998488/model_final_480dd8.pkl > output/cascade_mask_rcnn.txt

In [0]:
!cp detectron2_repo/output/cascade_mask_rcnn.txt /content/drive/My\ Drive/res_eval_redev

## Panoptic FPN

In [4]:
!pip install git+https://github.com/cocodataset/panopticapi.git

  Cloning https://github.com/cocodataset/panopticapi.git to /tmp/pip-req-build-0zfzf_oi
  Running command git clone -q https://github.com/cocodataset/panopticapi.git /tmp/pip-req-build-0zfzf_oi
  Created wheel for panopticapi: filename=panopticapi-0.1-cp36-none-any.whl size=8317 sha256=46bf05173c1c8c311bf4d6684286d234512986a9f8232c07b077a1b4661e66f0
  Stored in directory: /tmp/pip-ephem-wheel-cache-ds6czuy6/wheels/41/ae/17/f7e628e1ba4776ceea035aefb113ef24b6639ad5d5a2e5a5d1
Successfully built panopticapi


In [0]:
%%bash
cd detectron2_repo/datasets
mkdir coco
cd coco
cp /content/drive/My\ Drive/data_redev/coco/panoptic_annotations_trainval2017.zip .
unzip panoptic_annotations_trainval2017.zip
unzip annotations/panoptic_val2017.zip
cp /content/drive/My\ Drive/data_redev/coco/annotations/instances_val2017.json annotations/
cp /content/drive/My\ Drive/data_redev/coco/val2017.zip .
unzip val2017.zip

In [0]:
!cd detectron2_repo/datasets && python prepare_panoptic_fpn.py

### Panoptic R50-FPN

In [0]:
!cd detectron2_repo && python tools/train_net.py \
	--config-file configs/COCO-PanopticSegmentation/panoptic_fpn_R_50_3x.yaml \
	--eval-only MODEL.WEIGHTS detectron2://COCO-PanopticSegmentation/panoptic_fpn_R_50_3x/139514569/model_final_c10459.pkl > output/panoptic_r50_fpn.txt

In [0]:
!cp detectron2_repo/output/panoptic_r50_fpn.txt /content/drive/My\ Drive/res_eval_redev

### Panoptic R101-FPN

In [18]:
!cd detectron2_repo && python tools/train_net.py \
	--config-file configs/COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml \
	--eval-only MODEL.WEIGHTS detectron2://COCO-PanopticSegmentation/panoptic_fpn_R_101_3x/139514519/model_final_cafdb1.pkl > output/panoptic_r101_fpn.txt

model_final_cafdb1.pkl: 261MB [00:27, 9.34MB/s]               


In [0]:
!cp detectron2_repo/output/panoptic_r101_fpn.txt /content/drive/My\ Drive/res_eval_redev